In [1]:
"""將tensorflow檔轉成tensorflow lite，使機器負載量變小"""
from tensorflow import lite
from tensorflow.keras import models

# Parameters
keras_model_filename = './h5/recordingSpecAugment2_wen.h5' #訓練好的模型
tflite_filename = './tflite/recordingSpecAugment2_wen.tflite' #預建置檔案

# Convert model to TF Lite model
model = models.load_model(keras_model_filename) #載入本來的模型
converter = lite.TFLiteConverter.from_keras_model(model) #將模型載入轉換器
tflite_model = converter.convert() #進行轉換
open(tflite_filename, 'wb').write(tflite_model) #輸出轉換後的模型

INFO:tensorflow:Assets written to: C:\Users\user\AppData\Local\Temp\tmpdkyoquie\assets


501604

In [2]:
#slide_window=0.25
import sounddevice as sd
import numpy as np
import scipy.signal
import python_speech_features
import tensorflow as tf
from PyQt5.QtCore import QTime
import librosa

def sound(window,s,m):
    
    S = np.abs(librosa.stft(window)) #將整個window音訊做stft，並轉成絕對值
    
    if np.sum(S) > 3000: #判斷S的總和是否>3000，如果>3000，代表有講話
        # Compute features
        mfccs = python_speech_features.base.mfcc(window, #輸入訊號
                                            samplerate=sample_rate, #取樣率
                                            winlen=0.025, #音框涵蓋時間
                                            winstep=0.01, #音框間距離
                                            numcep=num_mfcc, #返回係數的量
                                            nfilt=26, #過濾器數量
                                            nfft=512,#FFT大小
                                            lowfreq=0, #梅爾濾波器最低頻率0(HZ)
                                            highfreq=None, #梅爾濾波器最高頻率:採樣率/2(HZ)
                                            preemph=0.97,  #不用預強化濾波器
                                            ceplifter=22,  #robust(不用)     
                                            appendEnergy=True,#True的話，第0個倒頻譜係數被替代成總音框能量的對數
                                            winfunc=np.hanning)#hanning window
        mfccs = mfccs.transpose()
        
        # Make prediction from model
        in_tensor = np.float32(mfccs.reshape(1, mfccs.shape[0], mfccs.shape[1], 1))
        #設定輸入張量
        interpreter.set_tensor(input_details[0]['index'], in_tensor)
        #進行預測
        interpreter.invoke()
        #取得輸出張量
        output_data = interpreter.get_tensor(output_details[0]['index'])
        
        val = output_data[0]#取得預測值
        val = val.tolist() #np.ndarray to list
        list_val_max = max(val) #取得最大值
        list_val_maxIndex = val.index(max(val)) #取得最大值的索引  
        
        
        if  list_val_max > 0.99 :
             data.append(str(m) + "分" + str(s) + "秒，出現:" + str(words[list_val_maxIndex]) + "，預測值:" + str(list_val_max))
            
#main
# Parameters
num_mfcc = 13 #回傳mfcc的量
model_path = './tflite/recordingSpecAugment2_wen.tflite'
words = ['backgroundNoise', 'ㄏㄧㄡ', 'ㄟ', '他', '你', '吼', '啦', '嗯', '好', '我', '的', '的一個', '的這個', '的那個', '著', '那', '那那個', '阿']#答案對應到的字詞
data = []
data_ho = []
start = 0 #一開始的索引值
end = 4000 #一開始的索引值
s = 0 #秒
m = 0 #分
duration = 180 #讀音檔的總時間
sample_rate = 16000 #取樣率

#載入音檔
y, sr = librosa.load("./Chen/chen_16000HZ.wav",sr=sample_rate,duration=duration) 

# Sliding window
window = np.zeros(8000)#取樣音頻數據變數

# Load model (interpreter)
interpreter = tf.lite.Interpreter(model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


while True:
    s = s + 0.25 #增加秒數
    if(s == 60): #60秒 轉成 1分
        s = 0
        m = m + 1

    window[:4000] = window[4000:] #把音訊載入window
    window[4000:] = y[start:end] #把音訊載入window
   
    sound(window,s,m) #呼叫sound()

    if(end == (16000 * duration)): #如果移動到最後，break
        break
    
    start = start + 4000 #向後移動
    end = end + 4000 #向後移動    
    
    
for i in range(len(data)):
        print(data[i])
        
print("總共:",len(data))


0分1.75秒，出現:嗯，預測值:0.9966939687728882
0分2.25秒，出現:你，預測值:0.9999687671661377
0分2.5秒，出現:你，預測值:0.9999146461486816
0分3.5秒，出現:你，預測值:0.996024489402771
0分5.0秒，出現:我，預測值:0.9999955892562866
0分6.5秒，出現:你，預測值:0.9962322115898132
0分6.75秒，出現:我，預測值:0.9995699524879456
0分8.0秒，出現:你，預測值:0.9945819973945618
0分8.25秒，出現:你，預測值:0.99997878074646
0分8.5秒，出現:你，預測值:0.9999992847442627
0分9.75秒，出現:你，預測值:0.999312162399292
0分10.0秒，出現:你，預測值:0.9999902248382568
0分10.5秒，出現:你，預測值:0.997713565826416
0分10.75秒，出現:的一個，預測值:0.9996378421783447
0分11.5秒，出現:嗯，預測值:0.9991638660430908
0分11.75秒，出現:你，預測值:0.9994866847991943
0分12.25秒，出現:嗯，預測值:0.9995680451393127
0分12.5秒，出現:你，預測值:0.9999998807907104
0分13.0秒，出現:我，預測值:0.9997411370277405
0分15.75秒，出現:你，預測值:0.9990957975387573
0分17.0秒，出現:你，預測值:0.9902783036231995
0分17.25秒，出現:我，預測值:0.9999740123748779
0分18.75秒，出現:你，預測值:0.9990178346633911
0分19.75秒，出現:你，預測值:0.9999324083328247
0分20.0秒，出現:你，預測值:0.9998365640640259
0分20.5秒，出現:你，預測值:0.9964094758033752
0分21.5秒，出現:你，預測值:0.9999234676361084
0分22.25秒，出現:你，預測值:0.9971225857